In [281]:
#import gensim
from gensim import corpora, models, similarities, utils
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis
from KAsql import query_SQL

In [282]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [283]:
# def call_mysql():
#     # pull in data from the reviews table
#     con = False
#     rows = np.nan
#     try:
#         con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
#         with con:
#             cur = con.cursor()
#             sql = 'USE yelpdata;'
#             cur.execute(sql)
#             sql = 'SELECT id,business_id,stars,comment FROM review;' 
#             #print(sql)
#             cur.execute(sql)
#             rows = cur.fetchall()
# #             for row in rows:
# #                 print(row)
#     except pymysql.Error, e:
#         print "Error %d: %s" % (e.args[0],e.args[1])
#         sys.exit(1)
#     finally:
#         if con:
#             con.close()
#     return rows  

In [285]:
# from KAsql import query_SQL
sql = 'SELECT id,business_id,stars,comment FROM review;' 
rows = query_SQL(sql)#returns a tuple within a tuple: each row is a tuple, inside one big tuple.

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

#print(documents[0])# prints one review

In [286]:
def canonicalize(word):    
    bads = ['dr.','dr',"n't","'ve","'re",'wo',"'m"]
    goods = ['doctor','doctor','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [287]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",
            'I','i','me','mine','my',"i'm","i've",
            'you','your','yours',"you're","you've",
            'it','its',"it's",
            'they','their','theirs',"they're","they've",
            'we','us','our','ours',"we're","we've"]
    few = ['male','male','male','female','female','female',
           'firstper','firstper','firstper','firstper','firstper','firstper','firstper',
           'secper','secper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [288]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [289]:
def ids_to_names():
# Purpose of function is to exclude proper names of doctors - first and last names
# Function will accept list of yelp_ids extracted by mysql from business table
# Function will put together a list of names
# Function will exclude terms like family doctor and pediatrics

    dashed_names=[]#examples = ['tamalpais-pediatrics-novato','julie-doctor-kim']
    sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
    yelp_ids = query_SQL(sql)# extracts unique yelp_ids
    for i in range(0,len(yelp_ids)):
        dashed_names.append(yelp_ids[i][0])
    
    def split_append(yelp_id):
        yelp_id = yelp_id.split('-')
        #print(yelp_id)
        return yelp_id

#     business_type = ['pediatrics','family','medicine','doctor','institute','center',
#                      'practice','md','san','francisco','ucsf','one','medical','group',
#                     'sports','fitness','therapy','acupuncture','chiropractic','district']
    
    names = []
#     for name in dashed_names:
#         [names.append(l) for l in split_append(name) if l not in business_type]
    for name in dashed_names:
        [names.append(l) for l in split_append(name)]
    return names

In [290]:
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    pat3 = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    #tokenizer = RegexpTokenizer(pat3)#time
    return pat3

In [293]:
# CLEAN & TOKENIZE - stoplists, lowercase, regular expressions, time-expressions

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
#document = a string

#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')#money
#tokenizer = RegexpTokenizer('\d*'+'\:'+'\d*')#time
tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money
mystopwords = stopwords.words() + ids_to_names() + ['dr.','dr','doctor','doctors',"doctor's","doctors'",'get']#type list
#print(mystopwords[-500:])


texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower().replace('.',' ')) if word not in mystopwords]
        for document in documents]#review in reviews 

In [294]:
# texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower()) if word not in stopwords.words()]
#         for document in documents]#review in reviews 

print(texts[:2])

[['adding', 'information', 'another', 'great', 'thing', 'willing', 'experiment', 'fibromyalgia', 'brought', 'book', 'specialist', 'willing', 'try', 'protocol', 'many', 'would', "won't", 'anything', 'crazy', 'consider', 'something', 'reasonable', 'made', 'available', 'emergency', 'severe', 'toothearache', 'offsite', 'seminar,', 'receptionist', 'asked', 'call', 'did,', 'prescribed', 'something', 'think', 'even', 'checked', 'later', 'called', 'back', 'either', 'way,', 'got', 'treatment', 'needed', 'think', 'takes', 'ahold', 'receptionistplease', 'keep', 'mind', 'receptionist', 'see', 'star', 'reviews,', 'spends', 'time', 'patients', 'little', 'patient', 'soon', 'benefit', 'great', 'treatment'], ["it's", 'tough', 'write', 'review,', "i've", 'patient', 'least', 'years', 'now,', "i'm", 'huge', 'personally,', "he's", 'always', 'given', 'sound', 'advice', 'easy', 'work', 'with,', 'recently', "he's", 'lost', 'personal', 'touch', 'result', 'best', 'rated', 'yelp', 'sf', 'goes', 'massive', 'popul

In [297]:
# BAG OF WORDS
dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
#dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/doctor.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(24041 unique tokens: [u'considered,', u'hanging', u'woody', u'issuescontroversies', u'specifics,']...)


In [298]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('doctor_bow_for_lda.mm', corpus) # store to disk, for later use

#print(corpus)
# looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

In [299]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=50, update_every=0, passes=10, iterations=100)


In [300]:
# SAVE THE LDA OBJECT
# --- note - i tried to do this with gensim, but it wasn't working, and ultimately cpickle did
#from gensim.utils import SaveLoad
#save(fname_or_handle, separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
#utils.SaveLoad.save(lda, pickle_protocol=2)

import cPickle as pickle
pickle_out = open('lda_50t.p', 'wb')
pickle.dump(lda, pickle_out)
pickle_out.close()

In [301]:
# PRINT THE TOPICS
lda.top_topics(corpus,num_words=10)

[([(0.0085843072242072129, u'skin'),
   (0.0082194887909188344, u"i've"),
   (0.0077446313597712103, u'would'),
   (0.0075752436606174346, u'see'),
   (0.0072934488067063433, u'office'),
   (0.0070056753195862744, u'never'),
   (0.0069972545480880303, u'time'),
   (0.0065816045275488731, u'go'),
   (0.0064658703723318809, u'back'),
   (0.0053809666494163081, u'years')],
  -80.87816680609248),
 ([(0.010228005613634234, u'surgery'),
   (0.0090861547884414882, u'time'),
   (0.0085492475635842759, u'back'),
   (0.008467282803816709, u'would'),
   (0.0069995066457993584, u'like'),
   (0.0064452366178624878, u'called'),
   (0.006028977874171052, u'go'),
   (0.0054770502324792683, u'could'),
   (0.0049602581536953476, u'told'),
   (0.0046170257326834559, u'see')],
  -81.971786719729948),
 ([(0.0098037145490895549, u"he's"),
   (0.0083901842261292995, u'office'),
   (0.0060317554351968956, u'back'),
   (0.0054879183062105904, u'see'),
   (0.0050180913144063687, u'time'),
   (0.0049210226961752

In [302]:
# GAMMAS MATRIX: REVIEWS BY TOPICS, topic distributions over each review, given as proportions of all topics within a review
gammas,_ = lda.inference(corpus)# first element returned by lda.inference is gammas matrix: #shape (3616,10)
# Normalize:  sum over topics should equal one
norm_gammas = gammas/(np.sum(gammas,axis=1)[:,None])# Axis=1 = sum over columns for each row; Axis=0 sum over rows in each column
print(norm_gammas[0][:])#row 0, all gammas
#print(sum(norm_gammas[0][:]))#Yes, I checked that =1 - normalization worked
norm_gammas_df = pd.DataFrame(norm_gammas)


[  3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   9.85151515e-01   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04   3.03030303e-04   3.03030303e-04
   3.03030303e-04   3.03030303e-04]


In [303]:
# Add the R-ID (primary key) and B-ID (foreign key) to the NORM-GAMMAS DF
sql = 'SELECT id,business_id FROM review'+';'
rows = query_SQL(sql)# returns shape #reviews x 2

# check that size are equivalent
if np.shape(norm_gammas_df)[0] !=np.shape(rows)[0]:
    print('Problem - rows in norm_gammas_df and sql query are not identical.')
else:
    #Convert sql return to array, and horizontally concatenate with gammas array, then convert to df
    ids_array = np.array(rows)
    ids_gammas = np.hstack((ids_array, norm_gammas))#combine ids with gammas in numpy array
    ids_gammas_df = pd.DataFrame(ids_gammas)
    #Rename columns (so topic # matches col) and confirm col-names, df.shape
    ids_gammas_df.columns = [np.arange(-1,ids_gammas_df.shape[1]-1)]
    ids_gammas_df = ids_gammas_df.rename(columns={-1: 'RID', 0: 'BID'})
    print(ids_gammas_df.columns)
    print(ids_gammas_df.shape)

Index([u'RID', u'BID',      1,      2,      3,      4,      5,      6,      7,
            8,      9,     10,     11,     12,     13,     14,     15,     16,
           17,     18,     19,     20,     21,     22,     23,     24,     25,
           26,     27,     28,     29,     30,     31,     32,     33,     34,
           35,     36,     37,     38,     39,     40,     41,     42,     43,
           44,     45,     46,     47,     48,     49,     50],
      dtype='object')
(3616, 52)


In [304]:
# SAVE THE NORM_GAMMAS DATAFRAME OBJECT

import cPickle as pickle
# pickle_out = open('norm_gammas_df.p', 'wb')
# pickle.dump(norm_gammas_df, pickle_out)
pickle_out = open('ids_gammas_df.p', 'wb')
pickle.dump(ids_gammas_df, pickle_out)
pickle_out.close()

In [305]:
# CALCULATE AVG topic distribution (gamma vector) for each BID (averaging over RID)
# Should this be a new program, and just import ids_gammas_df?  Yes I think

bid_gmeans = ids_gammas_df.drop('RID', 1)
bid_gmeans = bid_gmeans.groupby('BID',axis=0,as_index='BID').mean()
#bid_gmeans.columns = [np.arange(1,bid_gmeans.shape[1]+1)]# not needed, since I renamed columns of ids_gammas_df instead

# Kept for reference syntax/comments
###bid_gamma_means = bid_gamma_means.drop(bid_gamma_means.index[0])# No, this actually just deletes a real row of data, but it is the right syntax
###bid_gamma_means.shape[0]#number of rows.  Matches: mysql query:  select count(distinct(business_id)) from review;

# Print df to inspect
#print(bid_gamma_means.loc[:,:])# works but prints everything
#bid_gamma_means.head(5)
bid_gmeans.tail(5)

#bid_gmeans = bid_gmeans.sort([np.arange(1,(bid_gmeans.shape[1])+1)], ascending=False)#doesn't work
bid_gmeans = bid_gmeans.sort([1,2,3,4,5],ascending=False)
bid_gmeans.tail(5)

# SAVE (ADDED to .GITIGNORE)
# pickle_out = open('bid_gmeans.p', 'wb')
# pickle.dump(bid_gmeans, pickle_out)
# pickle_out.close()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
BID,,,,,,,,,,,,,,,,,,,,,
223,0.000383,0.024521,0.000383,0.003613,0.034819,0.000383,0.022340,0.000383,0.000383,0.000383,...,0.025125,0.000383,0.000383,0.024827,0.049650,0.000383,0.008315,0.039031,0.025160,0.025078
272,0.000330,0.000330,0.174296,0.032290,0.000330,0.000330,0.000330,0.031872,0.000330,0.101472,...,0.032334,0.000330,0.000330,0.000330,0.008098,0.000330,0.028920,0.000330,0.000330,0.000330
236,0.000327,0.049586,0.000327,0.000327,0.013584,0.074741,0.040378,0.024806,0.000327,0.051756,...,0.000327,0.009826,0.000327,0.025096,0.000327,0.024702,0.000327,0.000327,0.000327,0.000327
304,0.000305,0.000305,0.000305,0.000305,0.000305,0.000305,0.018224,0.000305,0.040534,0.000305,...,0.041450,0.000305,0.000305,0.000305,0.081270,0.000305,0.073803,0.000305,0.036512,0.000305
291,0.000269,0.000269,0.001935,0.024436,0.000269,0.010901,0.003461,0.000269,0.000269,0.000269,...,0.000269,0.012740,0.000269,0.000269,0.020186,0.024994,0.000269,0.000269,0.015857,0.000269


In [306]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)#html object
#Save figure object. 
#pickle.dump(vis_data, open('vis_data.p','wb'))